In [77]:
import json
import xml.etree.ElementTree as ET
from pathlib import Path

def load_experiment_metadata(input_path: str):

    config = json.load(open(input_path + "/config.json"))
    config["NETWORK"] = Path(config["NETWORK"])
    config["SUMO"] = Path(config["SUMO"])
    # [CO] updated simulation setup to use OD scenario #4: 42-dimensional  + low congestion.
    #sim_setup = json.load(open(input_path + "simulation_setups_co_gt.json"))
    sim_setup = json.load(open(input_path + "simulation_setups_lowCongestion_5minDemand.json"))

    return config, sim_setup


In [78]:

input_path = '/Users/rodrse/Downloads/calibrate_ods_carolina/network/SFO/metadata/'
config, sim_setup = load_experiment_metadata(input_path)

sim_setup

{'objective': 'counts',
 'net': 'SFO.net.xml',
 'route2od': 'trips24h_smoothed.rou.xml',
 'taz': 'taz.xml',
 'add': 'additional.add.xml',
 'start_sim_sec': 14400,
 'end_sim_sec': 15000,
 'current_od': 'current_od.xml',
 'init_od': 'initOD_lowCongestion_5minDemand.xml',
 'gt_od': 'initOD_lowCongestion_5minDemand.xml',
 'n_sumo_replicate': 1,
 'prefix_gt': 'gt',
 'prefix_sim_run': 'current'}

## Visualize effective tazes 

In [79]:
net_path = config['NETWORK']
tree = ET.parse(f'{net_path}' + "/" + sim_setup['gt_od'])
root = tree.getroot()

od_mat = []
for taz_elem in root.iter('tazRelation'):
    od_mat.append([taz_elem.get('from'), taz_elem.get('to'), int(taz_elem.get('count'))])


import pandas as pd
df_od = pd.DataFrame(od_mat, columns=['tazFrom', 'tazTo', 'tazCount'])


In [74]:
import numpy as np
effective_tazes = np.unique(np.concatenate([df_od.tazFrom.values, df_od.tazTo.values])).tolist()
len(effective_tazes)

1262

In [76]:
# same average hourly demand as the initial SFO OD but now set for 1hr of simulation
tree = ET.parse(f'{net_path}' + "/" + sim_setup['taz'])
root = tree.getroot()

for element in root.findall('taz'):
    if element.attrib['id'] not in effective_tazes:
        #print(element.attrib['id'])
        root.remove(element)

tree.write(f'{net_path}' + "/effective_taz_lowCongestion_1hrDemand.xml")

In [55]:
len(root.findall('taz'))

1300

Check how many routes enter and exit from SFO

In [57]:
net_path = config['NETWORK']
tree = ET.parse(f'{net_path}' + "/" + sim_setup['gt_od'])
root = tree.getroot()

od_mat = []
for taz_elem in root.iter('tazRelation'):
    od_mat.append([taz_elem.get('from'), taz_elem.get('to'), int(taz_elem.get('count'))])

import pandas as pd
df_od = pd.DataFrame(od_mat, columns=['tazFrom', 'tazTo', 'tazCount'])
df_od.shape

(45871, 3)

In [68]:
df_od.query("tazFrom  in ('613501','603900','604101')or tazTo in ('613501','603900','604101')")

,tazFrom,tazTo,tazCount
707,11000,604101,1
791,11100,604101,1
876,11200,604101,3
1302,11700,604101,1
1674,12100,604101,1
...,...,...,...
45711,980900,603900,1
45713,980900,604101,1
45836,984300,603900,1
45837,984300,604101,22


In [58]:
df_od.columns

Index(['tazFrom', 'tazTo', 'tazCount'], dtype='object')